In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import cmocean.cm as cmo
import xgcm
import MITgcmutils as umit
import gsw
import postmit as pm

In [3]:
path_to_data = "/home/jan/Data/MITgcm_test_data/nc_Diags/"
path_to_input = "/home/jan/Data/MITgcm_test_data/input/"
#path_to_data = "/path/to/data/"
#path_to_input = "/path/to/input/"

In [4]:
ds = xr.merge([xr.open_dataset(path_to_data + "2D_diags_1m_0222.nc"), xr.open_dataset(path_to_data + "3D_diags_1m_0222.nc"), xr.open_dataset(path_to_data + "layers_diags_1m_0222.nc")])

In [5]:
# modify ds so that it does not pass all checks
ds["YG"].attrs["c_grid_axis_shift"] = [-0.5, ]
ds = ds.rename({"layer_center": "_UNKNOWN_"})
ds = ds.assign_coords({'_UNKNOWN_': np.arange(1, 51)})

### Apply some checks
This will test  
    
1. Whether the attribute `c_grid_axis_shift` is a float. `c_grid_axis_shift` should be present for coordinates `XG`, `YG`, `Zl`, `Zp1`, `Zu`. When opening MITgcm output with `xmitgcm` and then sotring it as netcdf with `xarray.to_netcdf()` the attribute `c_grid_axis_shift` is put into a list (e.g. `[0.5, ]`) which cannot be read by `xgcm` and thus needs to be converted to a float (e.g. `0.5`).
2. Whether the z-distances `drW`, `drS`, `drC` are present in the dataset. These are needed for the metrics supplied to `xgcm`. If they are not present, they will be created from `drF` and `hFacW`, `hFacS`, `hFacC`, respectively.
3. If the simulation used the `Layers` package it will be checked if the coordinate for the layers is still called `_UNKNOWN_` (that's how it's called when loading data with `xmitgcm`). If so, it will be renamed `layer_centers` and populated with the values at the layer centers as derived from the `data.layers` file.

In [6]:
ds = pm.checks.apply_all_checks(ds, path_to_input)

In [7]:
ds

<xarray.Dataset>
Dimensions:       (YC: 300, XC: 250, time: 12, Z: 55, Zp1: 56, XG: 250,
                   YG: 300, Zl: 55, Zu: 55, layer_center: 50)
Coordinates: (12/37)
    Depth         (YC, XC) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    PHrefC        (Z) float32 dask.array<chunksize=(55,), meta=np.ndarray>
    PHrefF        (Zp1) float32 dask.array<chunksize=(56,), meta=np.ndarray>
  * XC            (XC) float32 5e+03 1.5e+04 2.5e+04 ... 2.485e+06 2.495e+06
  * XG            (XG) float32 0.0 1e+04 2e+04 ... 2.47e+06 2.48e+06 2.49e+06
  * YC            (YC) float32 5e+03 1.5e+04 2.5e+04 ... 2.985e+06 2.995e+06
    ...            ...
    rAz           (YG, XG) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    rhoRef        (Z) float32 dask.array<chunksize=(55,), meta=np.ndarray>
  * time          (time) object 0222-01-15 00:00:00 ... 0222-12-15 00:00:00
  * layer_center  (layer_center) float64 23.25 23.75 24.25 ... 28.35 28.45 28.75
    drW           (Z, YC, XG) float32 dask.array<chunksize=(55, 300, 250), meta=np.ndarray>
    drS           (Z, YG, XC) float32 dask.array<chunksize=(55, 300, 250), meta=np.ndarray>
Data variables: (12/33)
    ETAN          (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFempmr      (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFevap       (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFhl         (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFhs         (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFlwnet      (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    ...            ...
    VVEL          (time, Z, YG, XC) float32 dask.array<chunksize=(12, 55, 300, 250), meta=np.ndarray>
    WVEL          (time, Zl, YC, XC) float32 dask.array<chunksize=(12, 55, 300, 250), meta=np.ndarray>
    momHDiv       (time, Z, YC, XC) float32 dask.array<chunksize=(12, 55, 300, 250), meta=np.ndarray>
    LaHs1RHO      (time, layer_center, YG, XC) float32 dask.array<chunksize=(12, 50, 300, 250), meta=np.ndarray>
    LaVH1RHO      (time, layer_center, YG, XC) float32 dask.array<chunksize=(12, 50, 300, 250), meta=np.ndarray>
    LaVa1RHO      (time, layer_center, YG, XC) float32 dask.array<chunksize=(12, 50, 300, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Tue Dec 21 16:44:29 2021: ncks -O -4 -L 1 2D_diags_1m_0222....
    NCO:          netCDF Operators version 4.9.5 (Homepage = http://nco.sf.ne...